# Learned Solvent Embeddings + MLP

**Problem**: CV-LB gap is due to FIXED features that cannot capture solvent-specific patterns.

**Hypothesis**: By using learned embeddings for each solvent, the model can learn solvent-specific patterns during training, which could change the CV-LB relationship.

**Key Insight**: GNN benchmark achieved MSE 0.0039 by learning solvent representations. We approximate this with learned embeddings.

**Baseline**: exp_035 CV 0.008194, LB 0.0877

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

# Create solvent encoder
all_solvents = sorted(SPANGE_DF.index.tolist())
SOLVENT_ENCODER = {solvent: i for i, solvent in enumerate(all_solvents)}
print(f'Number of solvents: {len(SOLVENT_ENCODER)}')
print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Number of solvents: 26
Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Learned Embedding Model with Kinetics
class LearnedEmbeddingModel(nn.Module):
    def __init__(self, num_solvents=24, embedding_dim=32, kinetics_dim=5, data='single'):
        super().__init__()
        self.data_type = data
        self.embedding_dim = embedding_dim
        
        # Learnable embedding for each solvent
        self.solvent_embedding = nn.Embedding(num_solvents, embedding_dim)
        
        # Initialize embeddings with small random values
        nn.init.normal_(self.solvent_embedding.weight, mean=0, std=0.1)
        
        # Combine with kinetics features
        if data == 'single':
            input_dim = embedding_dim + kinetics_dim  # embedding + kinetics
        else:
            input_dim = 2 * embedding_dim + kinetics_dim + 1  # two solvents + kinetics + pct
        
        # MLP head
        self.net = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 3),
            nn.Sigmoid()
        )
    
    def forward(self, solvent_ids, kinetics, pct=None, solvent_b_ids=None):
        # Get solvent embeddings
        emb_a = self.solvent_embedding(solvent_ids)
        
        if self.data_type == 'single':
            x = torch.cat([emb_a, kinetics], dim=1)
        else:
            emb_b = self.solvent_embedding(solvent_b_ids)
            x = torch.cat([emb_a, emb_b, pct.unsqueeze(1), kinetics], dim=1)
        
        return self.net(x)

print('LearnedEmbeddingModel defined')

LearnedEmbeddingModel defined


In [5]:
# Wrapper class for training and prediction
class LearnedEmbeddingWrapper:
    def __init__(self, data='single', embedding_dim=32, n_models=5):
        self.data_type = data
        self.embedding_dim = embedding_dim
        self.n_models = n_models
        self.models = []
        self.solvent_encoder = SOLVENT_ENCODER
        
    def _get_kinetics(self, X):
        """Extract kinetics features: time, temp, 1/T, ln(t), interaction"""
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        return np.hstack([X_vals, inv_temp, log_time, interaction])
    
    def _encode_solvents(self, solvent_names):
        """Encode solvent names to integer IDs"""
        return np.array([self.solvent_encoder[s] for s in solvent_names])
    
    def train_model(self, X_train, y_train, epochs=300, batch_size=32, lr=1e-3):
        # Get kinetics features
        kinetics = torch.tensor(self._get_kinetics(X_train))
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'single':
            solvent_ids = torch.tensor(self._encode_solvents(X_train["SOLVENT NAME"].values))
            pct = None
            solvent_b_ids = None
        else:
            solvent_ids = torch.tensor(self._encode_solvents(X_train["SOLVENT A NAME"].values))
            solvent_b_ids = torch.tensor(self._encode_solvents(X_train["SOLVENT B NAME"].values))
            pct = torch.tensor(X_train["SolventB%"].values.astype(np.float64))
        
        # Train ensemble
        self.models = []
        for i in range(self.n_models):
            torch.manual_seed(42 + i)
            model = LearnedEmbeddingModel(
                num_solvents=len(self.solvent_encoder),
                embedding_dim=self.embedding_dim,
                kinetics_dim=5,
                data=self.data_type
            ).to(device).double()
            
            optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
            criterion = nn.HuberLoss()
            
            # Create dataset
            if self.data_type == 'single':
                dataset = TensorDataset(solvent_ids, kinetics, y_vals)
            else:
                dataset = TensorDataset(solvent_ids, solvent_b_ids, pct, kinetics, y_vals)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            model.train()
            for epoch in range(epochs):
                for batch in loader:
                    if self.data_type == 'single':
                        s_ids, kin, y = batch
                        s_ids, kin, y = s_ids.to(device), kin.to(device), y.to(device)
                        pred = model(s_ids, kin)
                    else:
                        s_a_ids, s_b_ids, p, kin, y = batch
                        s_a_ids = s_a_ids.to(device)
                        s_b_ids = s_b_ids.to(device)
                        p = p.to(device)
                        kin = kin.to(device)
                        y = y.to(device)
                        pred = model(s_a_ids, kin, p, s_b_ids)
                    
                    optimizer.zero_grad()
                    loss = criterion(pred, y)
                    loss.backward()
                    optimizer.step()
            
            model.eval()
            self.models.append(model)
    
    def predict(self, X_test):
        kinetics = torch.tensor(self._get_kinetics(X_test)).to(device)
        
        if self.data_type == 'single':
            solvent_ids = torch.tensor(self._encode_solvents(X_test["SOLVENT NAME"].values)).to(device)
            pct = None
            solvent_b_ids = None
        else:
            solvent_ids = torch.tensor(self._encode_solvents(X_test["SOLVENT A NAME"].values)).to(device)
            solvent_b_ids = torch.tensor(self._encode_solvents(X_test["SOLVENT B NAME"].values)).to(device)
            pct = torch.tensor(X_test["SolventB%"].values.astype(np.float64)).to(device)
        
        preds = []
        with torch.no_grad():
            for model in self.models:
                if self.data_type == 'single':
                    pred = model(solvent_ids, kinetics)
                else:
                    pred = model(solvent_ids, kinetics, pct, solvent_b_ids)
                preds.append(pred.cpu())
        
        return torch.clamp(torch.stack(preds).mean(dim=0), 0, 1)

print('LearnedEmbeddingWrapper defined')

LearnedEmbeddingWrapper defined


In [6]:
# Quick test on single fold
X_single, Y_single = load_data("single_solvent")
test_solvent = sorted(X_single["SOLVENT NAME"].unique())[0]
mask = X_single["SOLVENT NAME"] != test_solvent

model = LearnedEmbeddingWrapper(data='single', embedding_dim=32, n_models=3)
model.train_model(X_single[mask], Y_single[mask], epochs=100)
preds = model.predict(X_single[~mask])

actuals = Y_single[~mask].values
mse = np.mean((actuals - preds.numpy()) ** 2)
print(f'Test fold MSE: {mse:.6f}')
print(f'Predictions shape: {preds.shape}')

Test fold MSE: 0.080438
Predictions shape: torch.Size([37, 3])


In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = LearnedEmbeddingWrapper(data='single', embedding_dim=32, n_models=5)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y, epochs=300)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = LearnedEmbeddingWrapper(data='full', embedding_dim=32, n_models=5)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y, epochs=300)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE VERIFICATION ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nexp_035 baseline (fixed features): CV 0.008194')
print(f'This (learned embeddings): CV {overall_mse:.6f}')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_035!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_035')